In [65]:
#Import libraires
import spacy
import gensim
import nltk
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore")
import pyLDAvis
import pyLDAvis.gensim
import string
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split as tts, RandomizedSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from gensim.models import Word2Vec
from sklearn.metrics import f1_score
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [2]:
#Load and view dataset
df = pd.read_csv('clean_train4.csv')
df.head()

,tweet_id,sentiment,tokenized_text
0,1701,1,defining language of touch with different dial...
1,1851,1,learning ab doodle all doodle should be light ...
2,2689,2,one of the most in-your-face ex of stealing th...
3,4525,0,this would b pretty awesome if it did n't cras...
4,3604,1,outside the waiting for the


In [3]:
#Drop null values
df = df.replace(np.nan, '')

In [4]:
#Load hastag scores
hashtag_score = pd.read_csv('afinn_score.csv')

In [5]:
df_new = df.join(hashtag_score)
df_new = df_new.replace(np.nan, '')
df_new = df_new[df_new['sentiment'] != 3]

In [6]:
df_new.isnull().sum()

tweet_id          0
sentiment         0
tokenized_text    0
Unnamed: 0        0
0                 0
hashtags          0
dtype: int64

In [7]:
df_new['tokenized_text'] = df_new['tokenized_text'].replace(np.nan, '')

In [8]:
#Split into X and Y
X = df_new['tokenized_text']
y = df_new['sentiment']

In [9]:
#Initialize a tfidf vectorizer
cv = CountVectorizer(ngram_range=(1,3), min_df =2)
vector = cv.fit_transform(X)
X = vector.toarray()

In [10]:
#Make a dataframe
X_new = pd.DataFrame(X)

In [11]:
X_hashtags = np.array(df_new['0'])

In [12]:
X_new['Hashtag_score'] = X_hashtags

In [13]:
X_new['Hashtag_score'] = X_new['Hashtag_score'].replace('', 0)

In [14]:
for x in X_new['Hashtag_score']:
    float(x)

In [15]:
X_new['Hashtag_score'] = X_new['Hashtag_score'].abs()

In [16]:
#ratio to split into training and test set
ratio = int(len(df)*0.7)

In [17]:
#Split into training and test set
X_train, X_test, y_train, y_test = tts(X_new, y, random_state = 42, test_size = .025 )

In [18]:
#Initialize MultinomialNB model
nb = MultinomialNB(alpha = 0.5)

In [19]:
MultinomialNB?

In [20]:
#Fit model on training data
nb.fit(X_train, y_train)

MultinomialNB(alpha=0.5, class_prior=None, fit_prior=True)

In [21]:
#Make prediction using X_test
y_pred_nb = nb.predict(X_test)

In [22]:
#View accuracy metrics
accuracy_nb = accuracy_score(y_test, y_pred_nb)
print('The accuracy score using MultinomialNB model is {}.'.format(round(accuracy_nb,3)))
print(classification_report(y_test, y_pred_nb))

The accuracy score using MultinomialNB model is 0.676.
              precision    recall  f1-score   support

           0       0.39      0.50      0.44        14
           1       0.76      0.68      0.72       102
           2       0.64      0.71      0.68        63

    accuracy                           0.68       179
   macro avg       0.60      0.63      0.61       179
weighted avg       0.69      0.68      0.68       179



In [23]:
print(f1_score(y_test, y_pred_nb, average = 'weighted'))

0.6798280527062294


In [24]:
#Initialize LinearSVC
svc = LinearSVC(class_weight = 'balanced', random_state = 42, multi_class='crammer_singer')

In [25]:
#Fit model on training data
svc.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight='balanced', dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='crammer_singer', penalty='l2', random_state=42,
          tol=0.0001, verbose=0)

In [26]:
#Make prediction using X_test
y_pred_svc = svc.predict(X_test)

In [27]:
#View accuracy metrics
accuracy_svc = accuracy_score(y_test, y_pred_svc)
print('The accuracy score using LinearSVC model is {}.'.format(round(accuracy_svc,3)))
print(classification_report(y_test, y_pred_svc))

The accuracy score using LinearSVC model is 0.687.
              precision    recall  f1-score   support

           0       0.47      0.50      0.48        14
           1       0.76      0.73      0.74       102
           2       0.64      0.67      0.65        63

    accuracy                           0.69       179
   macro avg       0.62      0.63      0.62       179
weighted avg       0.69      0.69      0.69       179



In [28]:
print(f1_score(y_test, y_pred_svc, average = 'weighted'))

0.6886138352156909


In [48]:
rfc = RandomForestClassifier()

In [73]:
xgbc = XGBClassifier()

In [77]:
Model_List=[('naive bayes', nb),
            ('svc', svc),
           ('rfc', rfc),
           ('xgb', xgbc)]

In [78]:
voting_clf_soft = VotingClassifier(estimators = Model_List, voting = 'hard')

In [ ]:
voting_clf_soft.fit(X_train, y_train)

In [ ]:
y_pred_vc = voting_clf_soft.predict(X_test)
accuracy_vc = accuracy_score(y_test, y_pred_vc)
print(accuracy_vc)
print(classification_report(y_test, y_pred_vc))

In [ ]:
print(f1_score(y_test, y_pred_vc, average = 'weighted'))

In [272]:
svc2 = SVC(class_weight = 'balanced')

In [ ]:
svc2.fit(X_train, y_train)

In [ ]:
#Make prediction using X_test
y_pred_svc2 = svc.predict(X_test)

In [ ]:
#View accuracy metrics
accuracy_svc2 = accuracy_score(y_test, y_pred_svc2)
print('The accuracy score using LinearSVC model is {}.'.format(round(accuracy_svc2,3)))
print(classification_report(y_test, y_pred_svc2))

In [ ]:
print(f1_score(y_test, y_pred_svc2, average = 'weighted'))

In [54]:
#Load test set
test = pd.read_csv('clean_test2.csv')
test.head()
hashtag_test = pd.read_csv('afinn_score_test.csv')

In [55]:
test_new = test.join(hashtag_test)

In [56]:
test_new

,tweet_id,tokenized_text,Unnamed: 0,0,hashtags
0,7506,audience q what prototyping tool do you use sk...,0,0.0,myprototype sxsw
1,7992,at send your best to ...,1,0.0,citizen_journalism cnn ireport photography sxs...
2,247,and here a pic of you winning your cc cont,2,0.0,unsix sxsw
3,7688,marissa mayer phone a a cursor of physical loc...,3,0.0,sxsw
4,3294,is even cooler than i thought,4,0.0,SXSW
5,6125,front of popup at last night,5,0.0,SXSW
6,6131,my next life i 'm coming back a an 2. woman ca...,6,0.0,SXSW
7,4134,celebrating pi day style at,7,0.0,SXSW
8,8206,hmmm is it a bit weird that is not tending but...,8,0.0,sxsw
9,8552,to 'circles later today at got ta love one pla...,9,0.0,SXSW SXSW


In [57]:
#Drop null values from test
test_new = test_new.replace(np.nan, '')

In [58]:
#Vectorize test
vector_test = cv.transform(test_new['tokenized_text'])
X_test_clean = vector_test.toarray()

In [59]:
X_test_clean = pd.DataFrame(X_test_clean)

In [60]:
X_test_clean['afinn_score'] = test_new['0']

In [61]:
X_test_clean['afinn_score'].value_counts()

 0.0    1783
 4.0      11
 1.0       8
-2.0       8
 2.0       6
-1.0       2
 3.0       1
Name: afinn_score, dtype: int64

In [62]:
#Make prediction using X_test
y_test_svc = voting_clf_soft.predict(X_test_clean)

In [63]:
#Combine ID and validation Pred
ID = pd.DataFrame(test['tweet_id'], columns =['tweet_id'])
sentiment = pd.DataFrame(y_test_svc, columns = ['sentiment'])
submission = ID.join(sentiment)
print(submission)

      tweet_id  sentiment
0         7506          1
1         7992          1
2          247          1
3         7688          2
4         3294          2
5         6125          1
6         6131          2
7         4134          1
8         8206          1
9         8552          1
10        1634          1
11        4256          2
12        4921          1
13        1694          1
14        4193          1
15        7682          2
16        8151          2
17        1274          1
18        5868          1
19        3327          1
20          12          1
21        5839          1
22        5562          1
23        2959          1
24        8940          1
25        1995          1
26         607          2
27        7364          2
28        8167          2
29         681          1
...        ...        ...
1789      3798          1
1790      2587          1
1791      6709          2
1792      1731          1
1793      8690          1
1794       788          1
1795      38

In [64]:
#Save submission
submission.to_csv('submission voting.csv', index = False)